--------------------------------

In [1]:
import operator
from numpy import load
import faiss 

import pandas as pd
import numpy as np
import os 
import re

import nltk 
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer

In [2]:
df_hot=pd.read_csv("HotelsFinal.csv",index_col=0)
df_resto=pd.read_csv("RestaurantsFinal.csv",index_col=0)
df_att=pd.read_csv("AttractionsFinal.csv",index_col=0)
df_hot['id']=[str(i) for i in range(len(df_hot))]
del df_hot["dist"]
df_resto['id']=[str(i) for i in range(len(df_resto))] 
df_att['id']=[str(i) for i in range(len(df_att))]
df_hot=df_hot.fillna(" ")
for i in range(len(df_hot["Description"])) : 
    df_hot["Description"][i]=df_hot["Description"][i].replace('"',"'")
for i in range(len(df_hot["info"])) : 
    df_hot["info"][i]=df_hot["info"][i].replace('"',"'")
df_resto=df_resto.fillna(" ")
df_att=df_att.fillna(" ")
for i in range(len(df_att["description"])) : 
    df_att["description"][i]=df_att["description"][i].replace('"',"'")
for i in range(len(df_att["info"])) : 
    df_att["info"][i]=df_att["info"][i].replace('"',"'")
for i in range(len(df_resto["info"])) : 
    df_resto["info"][i]=df_resto["info"][i].replace('"',"'")    
for i in range(len(df_resto["ad_about"])) : 
    df_resto["ad_about"][i]=df_resto["ad_about"][i].replace('"',"'")
    
def search_bert(query,cat):
    
    if cat==1:  #Hotels
        query.replace("stars","")
        
        sbert_model = SentenceTransformer("sbert_model") 
        sentence_embeddings = load('bert_embeddings.npy')
        d=sentence_embeddings.shape[1]
        index = faiss.IndexFlatL2(d)   
        index.train(sentence_embeddings) 
        index.add(sentence_embeddings) 
        xb=sbert_model.encode([query])
        D,I=index.search(xb,1800)
        sentences=df_hot['info'].to_list()
        return D,I,sentences
    elif cat==2: #restaurants
        
        sbert_model = SentenceTransformer("sbert_model") 
        sentence_embeddings = load('rbert_embeddings.npy')
        d=sentence_embeddings.shape[1]
        index = faiss.IndexFlatL2(d)   
        index.train(sentence_embeddings) 
        index.add(sentence_embeddings) 
        xb=sbert_model.encode([query])
        D,I=index.search(xb,1200)
        sentences=df_resto['info'].to_list()
        return D,I,sentences
    
    elif cat==3:
        
        sbert_model = SentenceTransformer("sbert_model") 
        sentence_embeddings = load('abert_embeddings.npy')
        d=sentence_embeddings.shape[1]
        index = faiss.IndexFlatL2(d)   
        index.train(sentence_embeddings) 
        index.add(sentence_embeddings) 
        xb=sbert_model.encode([query])
        D,I=index.search(xb,58)
        sentences=df_att['info'].to_list()
        return D,I,sentences
def return_res(query,cat):
    if cat==1:
        D,I,sentences=search_bert(query,1)
        data=""
        for i in range(len(I[0])):
            if(i!=0):
                data+=","
            data+=str((I[0,i]))
        return str(data) 
    elif cat==2: 
        D,I,sentences=search_bert(query,2)
        data=""
        for i in range(len(I[0])):
            if(i!=0):
                data+=","
            data+=str((I[0,i]))
        return str(data) 
    elif cat==3: 
        D,I,sentences=search_bert(query,3)
        data=""
        for i in range(len(I[0])):
            if(i!=0):
                data+=","
            data+=str((I[0,i]))
        return str(data)  

In [3]:
from flask import Flask, jsonify,request 
from googletrans import Translator

app = Flask(__name__) 
# decorator to associate
# a function with the url 
@app.route("/searchHotel",methods=['POST']) 
def  searchHotel():
    if request.method == 'POST':    
        translator = Translator()
        query=request.form["message"]
        logha=request.form["logha"]
        if(logha=="English") : src1="en"
        if(logha=="French") : src1="fr"
        if(logha=="Arabic") : src1="ar"
        if(logha=="Spanish") : src1="es"
        translation=translator.translate(query, src=src1,dest='en')
        query=translation.text
        print(query) 
        res=return_res(query,1)
#         print(res)
        return res
    else:
        return "error" 
    
@app.route("/searchRestaurants",methods=['POST'])
def  searchRestaurant():
    if request.method == 'POST':    
        translator = Translator()
        query=request.form["message"]
        logha=request.form["logha"]
        if(logha=="English") : src1="en"
        if(logha=="French") : src1="fr"
        if(logha=="Arabic") : src1="ar"
        if(logha=="Spanish") : src1="es"
        translation=translator.translate(query, src=src1,dest='en')
        query=translation.text
        print(query) 
        res=return_res(query,2)
#         print(res)
        return res
    else:
        return "error"
@app.route("/searchAttractions",methods=['POST'])
def  searchAttractions():
    if request.method == 'POST':   
        translator = Translator() 
        query=request.form["message"] 
        logha=request.form["logha"]
        if(logha=="English") : src1="en"
        if(logha=="French") : src1="fr"
        if(logha=="Arabic") : src1="ar" 
        if(logha=="Spanish") : src1="es"
        translation=translator.translate(query, src=src1,dest='en') 
        query=translation.text 
        print(query) 
        res=return_res(query,3)
#         print(res) 
        return res
    else:
        return "error" 


@app.route("/rate",methods=['POST'])
def rate():
    df_rate=pd.read_csv("rateOurApp.csv",index_col=0)
    if request.method == 'POST':      
        rate =request.form["Rate"]
        username =request.form["username"]
        review =request.form["review"]
        print(username)
        print(rate)
        print(review)
        df_rate=df_rate.append({"Username":username,"Rate":rate,"Review":review},ignore_index=True)
        df_rate.to_csv("rateOurApp.csv")
        return rate
    else: 
        return "error" 
    
if __name__ == "__main__":
    app.run(host="0.0.0.0") 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Asian Restaurant


192.168.1.102 - - [28/Jun/2022 18:18:44] "POST /searchRestaurants HTTP/1.1" 200 -


Asian Restaurant


...

192.168.1.102 - - [28/Jun/2022 18:21:35] "POST /searchAttractions HTTP/1.1" 200 -


Asian Restaurant


192.168.1.102 - - [28/Jun/2022 18:22:10] "POST /searchRestaurants HTTP/1.1" 200 -


user
5.0
i like the app , but can you add comments 


192.168.1.102 - - [28/Jun/2022 18:23:35] "POST /rate HTTP/1.1" 200 -
